In [ ]:
# default_exp stockfish

# cheviz stockfish

This module integrates a chess engine, in this case the popular stockfish engine. The actual engine is provided by Python chess as [described here](https://python-chess.readthedocs.io/en/latest/engine.html). However, in our goal to build a complete ETL pipeline, we still need to get the engine running on our box. In a similar way as we got our data (PGN-encoded chess games), I'd like to use Jupyter Notebooks to perform the engine integration.

# 1. Motivation

For automated insights, we need a way to let the computer learn about our metrics. We need something to compare against our metrics, so I figured I start with stockfish, a very popular chess engine. Python chess also interfaces with stockfish so I won't need another dependency, either.

Stockfish is a small download and with the binaries having few dependencies they should be ready to go right after extracting the archive – no extra installation required. The engine should then help us by generating our training data.

# 2. Getting the engine

In [ ]:
!mkdir -p "tools"
!ls

00_core.ipynb	    cheviz	     index.ipynb  README.md	    tools
01_data.ipynb	    CONTRIBUTING.md  LICENSE	  requirements.txt
02_ui.ipynb	    data	     Makefile	  settings.ini
03_stockfish.ipynb  docs	     MANIFEST.in  setup.py


Assuming this notebook runs on a Linux box, we'll get the Linux binary [from the official website](https://stockfishchess.org/download/). We're not building high-end desktop software here (does such thing even exist?) so hard-coding the download URL for just the Linux version is OK. It might feel wrong, but the best advice to counter your (very valid) intuition is that we have to focus on our goal: automated insights. That is, don't spend time over-engineering the basic, non-data-sciency stuff in your pipeline *unless* your pipeline is ready to run in production and earn money for you. There is a reason why we use Python for all of this, so quick'n'dirty – to a certain degree – is the way to go.

We need to set a fake browser user-agent for our download request, otherwise we get 403'd. Normally that's a sign you're doing something wrong at extra costs to someone else (in this case ISP hosting fees for the stockfish communiy). At 1.7M of download size, which is probably smaller than a typical project's landing page these days, I don't feel guilty at all.

In [ ]:
#export
from pathlib import Path
import urllib.request
import shutil

def download():
    src = 'https://stockfishchess.org/files/stockfish-11-linux.zip'
    dst = Path().absolute() / 'tools' / 'stockfish.zip'
    
    if not dst.is_file():
        request = urllib.request.Request(src)
        request.add_header('Referer', 'https://stockfishchess.org/download/')
        request.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)')
        
        with urllib.request.urlopen(request) as response, open(dst, 'wb') as dst_file:
            shutil.copyfileobj(response, dst_file)
            
    return dst

We still need to extract the downloaded Zip archive. Also, we need to make the stockfish binary executable. There are 3 binaries available in this version, we use what I guess is the default one. More quick'n'dirty hardcoding that I expect to break sooner than later.

In [ ]:
#export
from pathlib import Path
import zipfile
import os
import stat

def extract(dst:Path):
    if not dst.is_file():
        return
    
    with zipfile.ZipFile(dst, 'r') as zip_file:
        zip_file.extractall(Path().absolute() / 'tools')
        dst_extracted = Path().absolute() / 'tools' / zip_file.namelist()[0]
            
        # make binary executable
        dst_binary = dst_extracted / 'Linux' / 'stockfish_20011801_x64'
        st = os.stat(dst_binary)
        os.chmod(dst_binary, st.st_mode | stat.S_IEXEC)
        
        return dst_binary

In [ ]:
stockfish = extract(download())

Check if we can run stockfish now, but let's not get stuck in stockfish's command prompt. So we send 'quit' to it immediately, which will still return the version of the binary and its authors.

In [ ]:
!{stockfish} quit

Stockfish 11 64 by T. Romstad, M. Costalba, J. Kiiski, G. Linscott
